<a href="https://colab.research.google.com/github/google/applied-machine-learning-intensive/blob/master/v2/06_other_models/02_embeddings/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Embeddings

Embeddings are a powerful way to represent data for deep learning models.

In this exercise we will work specifically with embeddings of words. This is one of the most common applications of this technique. We will train word embeddings from scratch on a small dataset and visualize those embeddings.

Embeddings can be trained using either supervised or unsupervised learning. For this exercise we will train a supervised sentiment classifier on IMDB movie reviews. We will learn our own word embeddings in the process.

## Setup

First we will set random seeds for reproducible results. Controlling which random numbers are generated ensures we get the same results every time we run this lab. **This should almost never be used in production code.**

In [0]:
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

## The Dataset

We will use the [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) that contains the text of 50,000 movie reviews from the [Internet Movie Database](https://www.imdb.com/). This dataset is conveniently packaged in TensorFlow Keras. 

In [0]:
from tensorflow import keras

imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data()

train_data.shape, train_labels.shape, test_data.shape, test_labels.shape

The reviews have been pre-processed, so the text of each review (a list of words) has been converted to a list of integers, and each integer represents a specific word.

In [0]:
print(train_data[0])

There are also some special placeholder characters. For instance, each review starts with a `start_char` placeholder that defaults to a value of `1`.

In [0]:
{review[0]: 'start_char' for review in train_data}

Each sentiment label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review. Thus this is a binary classification problem.

In [0]:
train_labels[0]

### Word Index

It is useful to be able to convert a list of integers back to text. The `imdb.get_word_index()` returns a dictionary of all of the words in the reviews. The words have a numeric value that represents their frequency in the dataset.

For instance, if we wanted to find the `25` most common words in the reviews, we could use the code below:

In [0]:
top_25 = ['' for _ in range(25)]
for k, v  in imdb.get_word_index().items():
  if v <= 25:
    top_25[v-1] = k

top_25

You can see that these words probably aren't helpful for predicting sentiment. The `skip_top=n` argument can be passed to [`load_data`](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data) to skip over the `n` most common words.

Let's reload our data using `skip_top`.

In [0]:
from tensorflow import keras

imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    skip_top=25,
)

train_data.shape, train_labels.shape, test_data.shape, test_labels.shape

Our data now doesn't include the top `25` words in the reviews. That begs the question: what happens to these words? Are they removed? Are the replaced?

It turns out they are replaced by a placeholder value. This value is the `oov_char`, which by default is `2`.

If we print out the first training review we can see that the top `25` most common words are now replaced with `2`, including the `start_char`!

In [0]:
train_data[0]

Let's create a function that can convert a numeric review into text so we can read the reviews ourselves.

To do this we need to modify the word index a bit. The index from Keras starts at `1`, but we know that the `start_char` is `1` and that the `oov_char` is `2`. We need to shift the indexes by a few values.

The `load_data` function has an argument called `index_from` that defaults to `3`. The reviews are indexed with the first word starting at `3`, while the word index has the first word starting at `1`. Let's fix that and add placeholders for start, oov, and a padding character we'll use to make the reviews equal-sized for modeling.

In [0]:
# A dictionary mapping each word to an integer index.
word_index = imdb.get_word_index()

# Reserve the first 3 indices for special tokens.
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<OOV>"] = 2

vocab_size = len(word_index)
print('# unique words: {}'.format(vocab_size))

And now let's try to decode a review.

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(train_data[0])

### Preprocessing

Movie reviews can be different lengths, but inputs to a neural network must all be the same length. Let's take a look at the distribution of review lengths.

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))
review_lengths = [len(review) for review in train_data]
plt.hist(review_lengths, density=True, cumulative=True)
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.show()

Even though the longest review is close to `2500` words long, many of the reviews have fewer than 500 words. 

We can use the `pad_sequences` function to standardize the lengths of the reviews to `500` words long. Any reviews longer than this will have the extra words truncated, while any reviews shorter than `500` words will have extra `"<PAD>"` tokens added to the end. 

Choosing a standardized length requires balancing efficiency (longer lengths mean slower training) and information loss (shorter lengths may truncate too much valuable information). Aiming for a length that fully covers `90%` of samples is generally reasonable, and you can further tune this as a hyperparameter.

In [0]:
maxlen = 500

train_data = keras.preprocessing.sequence.pad_sequences(
    train_data, value=word_index["<PAD>"], padding='post', maxlen=maxlen)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data, value=word_index["<PAD>"], padding='post', maxlen=maxlen)

Let's inspect the first padded review.

In [0]:
print(decode_review(train_data[0]))

## Build the Model

### Using the Embedding Layer

Keras makes it easy to use embeddings. The [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer takes at least two arguments: the number of possible words in the vocabulary and the dimensionality of the embeddings. We will start by using a small embedding size of 2 to make visualization easier.

In [0]:
embedding_dim = 2

embedding_layer = keras.layers.Embedding(vocab_size, embedding_dim)

The Embedding layer can be understood as a lookup table that maps from integer indices, which stand for specific words, to dense vectors (their embeddings). The dimensionality, or width, of the embedding is a parameter you can experiment with to see what works well for your problem, much in the same way you would experiment with the number of neurons in a Dense layer.

When we create an Embedding layer, the weights for the embedding are randomly initialized just like any other layer. During training, they are gradually adjusted via backpropagation. Once trained, the learned word embeddings will roughly encode similarities between words (as they were learned for the specific problem our model is trained on).

As input, the Embedding layer takes a 2D tensor of integers, of shape `(num_samples, sequence_length)`, where each sample is a sequence of integers. As output, the embedding layer returns a 3D floating point tensor, of shape `(num_samples, sequence_length, embedding_dimensionality)`. 

### Configure the Model

1. The first layer is an Embedding layer. This layer takes the integer-encoded vocabulary and looks up the embedding vector for each word index. These vectors are learned as the model trains. The vectors add a dimension to the output array. The resulting dimensions are: `(batch size, sequence length, embedding size)`.

1. Next, we flatten the output from a 2-d array to a 1-d array.

1. The last layer is densely connected with a single output node. Using the sigmoid activation function, this value is a float between 0 and 1, representing a probability, or confidence level, that the review is positive.

In [0]:
model = keras.Sequential([
  keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
  keras.layers.Flatten(),
  keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

### Compile and Train the model

In [0]:
model.compile(
  # Calculate loss for a binary classification problem.
  loss='binary_crossentropy',

  # Adam is one of the most commonly used optimizers.
  optimizer=tf.keras.optimizers.Adam(),

  # We will only track accuracy for this task.
  metrics=['accuracy']
)

The number of epochs and batch size used during training are both hyperparameters, which means you may need to experiment with different values to improve performance. This also means there's no magic answer for which values to choose.

If you've implemented your model correctly, you should observe decreasing training loss within a few epochs. With more epochs (more training), the model will experience more overfitting. You will typically train for at least 5 epochs. For this model, we train for 10 epochs as a reasonable tradeoff between learning (as the validation accuracy is still increasing) and overfitting.

Since we have a fairly large dataset, we also want to process data in batches instead of the entire dataset at once. If we choose batch sizes that are too small, we will get slower training per epoch, while larger batch sizes may require more epochs to train and even cause out-of-memory errors. It's good practice to start with a small power of two (e.g., 32), and then experiment with continuously doubling your batch size. For this model, we are able to use a batch size of 512 with good performance.

In [0]:
history = model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=512,
    validation_split=0.2
)

Let's visualize the training and validation accuracy over time.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))

plt.show()

With this approach, our model reaches a validation accuracy of around 87%. (Note that the model is already beginning to overfit, as reflected in the diverging training and validation accuracy curves.)

## Visualize Embeddings

### Retrieve the Learned Embeddings

Next, let's retrieve the word embeddings learned during training. This will be a matrix of shape `(vocab size, embedding dimension)`.

In [0]:
e = model.layers[0]
embedding_matrix = e.get_weights()[0]
print(embedding_matrix.shape)

Looking at all 80,000+ at once would be overwhelming, so let's explore the learned word embeddings on a small set of terms. Most of these terms are generally strongly indicative of sentiment, but some are added just for fun.

In [0]:
informative_terms = [ "bad", "great", "best", "worst", "fun", "beautiful",
                      "excellent", "poor", "boring", "awful", "terrible",
                      "definitely", "perfect", "liked", "worse", "waste",
                      "entertaining", "loved", "unfortunately", "amazing",
                      "enjoyed", "favorite", "horrible", "brilliant", "highly",
                      "simple", "annoying", "today", "hilarious", "enjoyable",
                      "dull", "fantastic", "poorly", "fails", "disappointing",
                      "disappointment", "not", "him", "her", "good", "time",
                      "sad", "exciting", "slow", "movie", "film", "action",
                      "comedy", "drama", "fabulous"]

Now we plot each word in `informative_terms` on a two-dimensional graph. Recall that we are using two-dimensional embeddings, so consider the first value as an x-value and the second value as a y-value. 

*Note: If the displayed plot is too small, try running the cell again.*

In [0]:
for word in informative_terms:
  word_num = word_index[word]
  embeddings = embedding_matrix[word_num]
  plt.text(embeddings[0], embeddings[1], word)

# Do a little set-up to make sure the plot displays nicely.
plt.rcParams["figure.figsize"] = (25, 25)
plt.xlim(1.2 * embedding_matrix.min(), 1.2 * embedding_matrix.max())
plt.ylim(1.2 * embedding_matrix.min(), 1.2 * embedding_matrix.max())
plt.show()

We can see that positive words tend to cluster around each, other and negative words tend to cluster around each other.

## Final Test

Let's see how our model performs on the holdout dataset.

In [0]:
predictions = model.predict(test_data)
predictions

The prediction values that we get are floating point numbers, but we want to convert these to a prediction of `0` for negative and `1` for positive. One way to do this is to simply round.

In [0]:
predictions = [int(round(prediction)) for prediction in predictions.flatten()]
predictions[:10]

And now we can measure our model quality.

In [0]:
precision = tf.keras.metrics.Precision()
precision.update_state(test_labels, predictions)

recall = tf.keras.metrics.Recall()
recall.update_state(test_labels, predictions)

print('Precision: {}, Recall: {}'.format(
    precision.result().numpy(), recall.result().numpy()))

# Exercises

## Exercise 1

What structures do you see in the embeddings of the words that were visualized earlier in this lab?

### **Student Solution**

*Your answer here:*



---

### Answer Key

There are many possible solutions. Here is an example.

> *There is a definite cluster of negative words, but the positive words are a little more spread out. The word 'sad' being in the middle is interesting. It would be a good exercise to get samples of positive and negative sentiment reviews that have the word 'sad' in them to see the context of how the word is used.*

---

## Exercise 2: Higher-Dimensional Embeddings

Now we can visualize two-dimensional embeddings, but recall that our original vocabulary had over 80,000 words; we're missing a lot of information if we only use two dimensions!

Typical embedding dimensions are 50, 100, 200, 300, and sometimes even larger.

Retrain our earlier model with the same hyperparameters, using **50-dimensional** embeddings and retrieve the learned embeddings from this trained model. You should observe a slower training time and much higher training accuracy after 10 epochs.

Print out the precision and recall on the testing holdout data.

### **Student Solution**

In [0]:
# Your code goes here

---

### Answer Key

In [0]:
# Re-train model using 50-dimensional embeddings.

embedding_dim = 50

model = keras.Sequential([
  keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
  keras.layers.Flatten(),
  keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
  loss='binary_crossentropy',
  optimizer=tf.keras.optimizers.Adam(),
  metrics=['accuracy']
)

history = model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=512,
    validation_split=0.2
)

predictions = model.predict(test_data)

precision = tf.keras.metrics.Precision()
precision.update_state(test_labels, predictions)

recall = tf.keras.metrics.Recall()
recall.update_state(test_labels, predictions)

print('Precision: {}, Recall: {}'.format(
    precision.result().numpy(), recall.result().numpy()))

---

## Exercise 3: Embedding Projector

In this section we'll introduce the [Embedding Projector](http://projector.tensorflow.org/), a tool to visualize high-dimensional embeddings.

We will now write the embeddings to disk. To use the Embedding Projector, we will upload two files in tab separated format: a file of vectors (containing the embeddings) and a file of metadata (containing the words). We will again only analyze our small set of terms from above.

Run the code block below to create the files necessary for the Embedding Projector.

In [0]:
import io

e = model.layers[0]
embedding_matrix = e.get_weights()[0]

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word in informative_terms:
  word_num = word_index[word]
  embeddings = embedding_matrix[word_num]

  out_m.write(word.encode("utf-8").decode("utf-8")  + "\n")
  out_v.write('\t'.join(
      [str(x).encode("utf-8").decode("utf-8") for x in embeddings]) + "\n")

out_v.close()
out_m.close()

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')




Finally, open the [Embedding Projector](http://projector.tensorflow.org/):

*   Click on "Load" in the Data panel.
*   Upload the two files we created above: `vecs.tsv` and `meta.tsv`.

The embeddings we have trained will now be displayed. You can search for words to find their closest neighbors.

*Note: Your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer.*

### Question 1

How do the structures of these embeddings compare to the two-dimensional embeddings we visualized earlier?

**Student Solution**

> *Your solution goes here*

---

#### Answer Key

There are many acceptable answers. Below is an example.

> *The structures seem to be very similar, with positive words in a cluster and negative words in another cluster. In the higher-dimensional model, the words seem to cluster tighter together, with only a few words floating around without peers.*

---

### Question 2

List at least `5` words that seem to add little value to the sentiment analysis.

**Student Solution**

> 1. *Your answer here*
> 1. *Your answer here*
> 1. *Your answer here*
> 1. *Your answer here*
> 1. *Your answer here*

#### Answer Key

Some of the words that seemed less useful in our model were:

> 1. time
> 1. film
> 1. her
> 1. him
> 1. not
> 1. drama
> 1. comedy
> 1. movie

---